# Interactively trigger GASpy tasks

- Wanted a notebook to interactively try GASpy calculations and tasks.

In [ ]:
import os
import glob
import pickle

from ase.visualize import view
import tqdm
import matplotlib.pyplot as plt
import subprocess
import datetime
import multiprocessing

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%load_ext blackcellmagic
%load_ext autoreload
%autoreload 2

In [ ]:
def load_from_file(fname, module=pickle):
    """
    Quick function to load an object from a file.

    By default, use the pickle module.
    """

    with open(fname, "rb") as f:
        obj = module.load(f)

    return obj

def save_to_file(obj, fname, module=pickle):
    """
    Quick function to save a single object to a file.

    By default, use the pickle module.
    """

    with open(fname, "wb") as f:
        module.dump(obj, f)

    return True

    # try:

    # except:
    #     print("Something went wrong with the pickling.")

## Routine tasks

- Here are some examples of tasks we'll routinely have to perform with GASpy.
- Some of these come from example scripts.

### Catalog population

- Tip: run this in a new console for the notebook. That way, the output will page automatically and you won't have a huge cell to clear at the end.

In [ ]:
'''
This script will populate your `catalog` Mongo collection with adsorption sites
of alloys containing the given set of elements and with Miller indices no
higher than the specified `max_miller`.
'''

from gaspy.tasks.db_managers.catalog import update_catalog_collection


#elements = ['Ag', 'Al', 'As', 'Au', 'Bi', 'Ca', 'Cd', 'Cl', 'Co', 'Cr', 'Cs',
#            'Cu', 'Fe', 'Ga', 'Ge', 'H', 'Hf', 'Hg', 'In', 'Ir', 'K', 'Mn',
#            'Mo', 'N', 'Ni', 'Na', 'Nb', 'Os', 'P', 'Pb', 'Pd', 'Pt', 'Rb',
#            'Re', 'Rh', 'Ru', 'S', 'Sb', 'Sc', 'Se', 'Si', 'Sn', 'Sr', 'Ta',
#            'Tc', 'Te', 'Ti', 'Tl', 'V', 'W', 'Y', 'Zn', 'Zr']
max_miller = 2

elements = ['Pt', 'Ru', "Cu", "Ni", "Pd"]


update_catalog_collection(elements=elements, max_miller=max_miller, n_processes=8)

In [ ]:
'''
Alternative form to run arbitrary functions within gaspy.tasks.db_managers.catalog
'''

import gaspy.tasks.db_managers.catalog as catalog


#elements = ['Ag', 'Al', 'As', 'Au', 'Bi', 'Ca', 'Cd', 'Cl', 'Co', 'Cr', 'Cs',
#            'Cu', 'Fe', 'Ga', 'Ge', 'H', 'Hf', 'Hg', 'In', 'Ir', 'K', 'Mn',
#            'Mo', 'N', 'Ni', 'Na', 'Nb', 'Os', 'P', 'Pb', 'Pd', 'Pt', 'Rb',
#            'Re', 'Rh', 'Ru', 'S', 'Sb', 'Sc', 'Se', 'Si', 'Sn', 'Sr', 'Ta',
#            'Tc', 'Te', 'Ti', 'Tl', 'V', 'W', 'Y', 'Zn', 'Zr']
max_miller = 2

elements = ['Pt', 'Ru', "Cu", "Ni", "Pd"]


get_mpid_task = catalog._GetMpids(elements=elements)

In [ ]:
catalog.schedule_tasks([get_mpid_task])

In [ ]:
mpids = catalog.get_task_output(get_mpid_task)

In [ ]:
mpids

In [ ]:
# Aside: searching for missing calculation
from gaspy.tasks import calculation_finders
from gaspy import defaults

In [ ]:
find_bulk_task = calculation_finders.FindBulk(mpid="mp-1186117")

In [ ]:
schedule_tasks([find_bulk_task])

In [ ]:
output = catalog.get_task_output(find_bulk_task)

### Updating collections

- Tip: run this in a new console for the notebook. That way, the output will page automatically and you won't have a huge cell to clear at the end.

In [ ]:
'''
This script will populate your `atoms` Mongo collection with completed
calculations in your FireWorks database.
'''
from gaspy.tasks.db_managers import update_all_collections

update_all_collections(n_processes=4)

### Triggering adsorption calculations

In [ ]:
all_site_documents = get_catalog_docs()

In [ ]:
from gaspy.tasks import schedule_tasks
from gaspy.gasdb import get_catalog_docs
from gaspy.tasks.metadata_calculators import CalculateAdsorptionEnergy


# Get all of the sites that we have enumerated
all_site_documents = get_catalog_docs()

# Pick the sites that we want to run. In this case, it'll be sites on
# palladium (as per Materials Project ID 2, mp-2) on (111) facets.
site_documents_to_calc = [doc for doc in all_site_documents
                          if (doc['mpid'] == 'mp-33' and
                              doc['miller'] == [1, 1, 1])]

# Turn the sites into GASpy/Luigi tasks
tasks = [CalculateAdsorptionEnergy(adsorbate_name='H',
                                   adsorption_site=doc['adsorption_site'],
                                   mpid=doc['mpid'],
                                   miller_indices=doc['miller'],
                                   shift=doc['shift'],
                                   top=doc['top'])
         for doc in site_documents_to_calc]

# Schedule/run all of the tasks
schedule_tasks(tasks)

### Use reservation system to give unit cell optimizations more walltime

In [ ]:
from gaspy import fireworks_helper_scripts

In [ ]:
# Look for RESERVED jobs that are unit cell optimizations
lpad = fireworks_helper_scripts.get_launchpad()

In [ ]:
fwids = lpad.get_fw_ids(
    query={"state": "RESERVED", "name.calculation_type": "slab+adsorbate optimization"}
)

In [ ]:
# Extract the Slurm job IDs from the Firework record
jobids = []

for fwid in fwids:
    fw = lpad.get_fw_by_id(fwid)
    try:
        jobids.append(fw.launches[-1].state_history[-1]["reservation_id"])
    except KeyError:
        print(f"""Could not get job ID for firework {fw.fw_id}""")

In [ ]:
len(jobids)

In [ ]:
# Ask Slurm to give all the jobs more time
# Hopefully 8 hrs is enough for a unit cell optimization to finish!!
cmd = f"""scontrol update jobid={",".join(jobids)} TimeLimit=48:00:00 TimeMin=8:30:00"""

In [ ]:
# Ask Slurm to lower the priority on GASpy jobs, so I can do calculations
# for other projects as well.
cmd = f"""scontrol update jobid={",".join(jobids)} Nice=100"""

In [ ]:
cmd

In [ ]:
result = subprocess.run(cmd.split())

In [ ]:
result

### Use reservation system to allocate more walltime for slabs with large numbers of atoms

In [ ]:
from gaspy import fireworks_helper_scripts
from gaspy import vasp_functions
from collections import Counter
import pandas as pd
import multiprocessing

In [ ]:
lpad = fireworks_helper_scripts.get_launchpad()

In [ ]:
fwids = lpad.get_fw_ids(query={"state": "FIZZLED"})

In [ ]:
len(fwids)

In [ ]:
fwids[-1]

In [ ]:
def hex_to_atoms(hex_: str):
    """Quickly get atoms corresponding to hex string"""
    fname = str(vasp_functions.uuid.uuid4()) + ".traj"
    vasp_functions.hex_to_file(fname, hex_)
    slab = vasp_functions.ase.io.read(fname)
    os.remove(fname)
    return slab

In [ ]:
# For each fwid, we want to get the associated Firework, get
# the atoms object associated with it, and then see what we're looking at.
def get_slab(fwid):
    lpad_tmp = fireworks_helper_scripts.get_launchpad()
    fw = lpad_tmp.get_fw_by_id(fwid)
    hex_ = fw.spec["_tasks"][1]["args"][-1]
    slab = hex_to_atoms(hex_)
    return slab


with multiprocessing.Pool(processes=16) as p:
    slabs = list(tqdm.tqdm(p.imap(get_slab, fwids), total=len(fwids)))

counts = [dict(Counter(slab.get_chemical_symbols())) for slab in slabs]

In [ ]:
view(slabs)

In [ ]:
df_counts = pd.DataFrame(data=counts)

In [ ]:
lengths = [len(slab) for slab in slabs]

In [ ]:
import seaborn as sns

In [ ]:
sns.swarmplot(data=lengths)

### Rerun fizzled fireworks that simply timed out

In [ ]:
from gaspy import fireworks_helper_scripts

In [ ]:
lpad = fireworks_helper_scripts.get_launchpad()

In [ ]:
# Get list of fizzled fireworks
fwids = lpad.get_fw_ids(query={"state": "FIZZLED"})

In [ ]:
len(fwids)

In [ ]:
fw = lpad.get_fw_by_id(fwids[240])

In [ ]:
directory = fw.launches[-1].launch_dir

In [ ]:
# Find the force trajectory
outcar_path = os.path.join(directory, "OUTCAR")

In [ ]:
result = subprocess.run(["grep", "FORCES: max", outcar_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE, encoding="utf8")

In [ ]:
# Parse grep output to find forces
lines = result.stdout[:-1].split("\n")
max_forces = np.array([line[24:36] for line in lines], dtype=np.float64)

In [ ]:
plt.plot(max_forces[-30:], "ko")
plt.grid()

In [ ]:
fw.launches[-1].state_history

In [ ]:
for fwid in fwids:
    fw = lpad.get_fw_by_id(fwid)
    try:
        # Firework has an exception. Don't rerun.
        exception = fw.launches[-1].action.stored_data["_exception"]
    except (KeyError, AttributeError):
        print(f"Firework {fwid} failed without exception; rerunning.")

        # Now check to see if

        # lpad.rerun_fw(fw_id=fwid)

In [ ]:
fw.launches[-1].action.stored_data["_exception"]

## Introspection

- Contains recipes for inspecting things in the database and getting an idea of what's going on.

In [ ]:
from gaspy import fireworks_helper_scripts

### View atoms that match some Fireworks query

In [ ]:
lpad = fireworks_helper_scripts.get_launchpad()

In [ ]:
completed_fwids = lpad.get_fw_ids(query={"state": "COMPLETED", "name.adsorbate": "N", "name.mpid": "mp-126"})

In [ ]:
completed_fwids[-1]

In [ ]:
completed_atoms = [fireworks_helper_scripts.get_atoms_from_fwid(fwid=fwid) for fwid in tqdm.tqdm(completed_fwids)]

In [ ]:
view(completed_atoms)

### Get initial geometries of completed adsorption calculations

In [ ]:
from gaspy.gasdb import get_adsorption_docs
from gaspy.fireworks_helper_scripts import get_atoms_from_fwid
from ase.io import write

In [ ]:
ads_docs = get_adsorption_docs(
    adsorbate="H", extra_projections={"initial_fwid": "$fwids.slab+adsorbate"}
)

In [ ]:
# Get atomic geometries
def lookup_initial_geometry(doc):
    """Wrapper to find initial geometry given an adsorption doc"""

    # Insert initial slab geometry into the document
    # and track success of operation
    new_doc = doc.copy()

    try:
        new_doc.update(
            {
                "initial_atoms": get_atoms_from_fwid(doc["initial_fwid"]),
                "id": str(doc["mongo_id"]),
                "success": True,
            }
        )
    except:
        new_doc.update({"id": str(doc["mongo_id"]), "success": False})

    return new_doc


# Parallelize to save time
with multiprocessing.Pool(processes=16) as p:
    results = list(
        tqdm.tqdm(p.imap(lookup_initial_geometry, ads_docs), total=len(ads_docs),)
    )

In [ ]:
# Show how many failed, if any
num_invalid = len(list(filter(lambda x: not x["success"], results)))
print(f"{num_invalid} initial geometries could not be found")

In [ ]:
# Now, save them all to CIF format

parent_dir = "exported-cifs"
os.makedirs(name=parent_dir, exist_ok=True)

def save_to_cif(doc):
    """Wrapper to save atom geometry to CIF format"""

    try:
        # Use the Mongo record ID as a unique hash
        fname = os.path.join(parent_dir, "slab_" + doc["id"] + ".cif")
        doc["initial_atoms"].write(fname)
        success = True
    except:
        success = False

    return {"id": doc["id"], "cifwrite_success": success}


# Parallelize to save time
with multiprocessing.Pool(processes=16) as p:
    write_results = list(tqdm.tqdm(p.imap(save_to_cif, results), total=len(results),))

In [ ]:
# Show how many failed, if any
num_valid = len(list(filter(lambda x: x["cifwrite_success"], write_results)))
print(f"{num_valid} initial geometries successfully written")

### Check status of Fireworks jobs

In [ ]:
job_status = fireworks_helper_scripts.check_jobs_status(user_ID="abhi2101", num_jobs=50)

In [ ]:
launch_dirs = job_status[job_status["state"]=="RUNNING"]

In [ ]:
launch_dirs

In [ ]:
for run in launch_dirs:
    print(run["fwid"])
    !ls -lah {run["launch_dir"]}

In [ ]:
job_status

In [ ]:
job_status.pivot_table(values="fwid", index="adsorbate", columns="shift", aggfunc="count")

### Re-run defused bulk optimization fireworks and delete old pickle files

- EXPERIMENTAL, not known to work. Do not use.

In [ ]:
# Need to get rid of unit cell optimization pickles that never finished
defused_fwids = lpad.get_fw_ids(
    query={
        "state": "RUNNING",
        "name.calculation_type": "unit cell optimization",
        "created_on": {"$gt": "2020-07-03"},
    }
)

# Now, for each FW, get the corresponding mpid

In [ ]:
for fwid in defused_fwids:
    lpad.defuse_fw(fwid)

In [ ]:
mpids = set([lpad.get_fw_by_id(fwid).name["mpid"] for fwid in defused_fwids])

In [ ]:
mpids

In [ ]:
# Rerun these bulk calculations
from gaspy.tasks import calculation_finders

find_bulk_tasks = [calculation_finders.FindBulk(mpid=mpid) for mpid in mpids]
schedule_tasks(find_bulk_tasks)

### Inspect adsorption energies

In [ ]:
import gaspy.gasdb as gasdb
import seaborn as sns

In [ ]:
sns.set_style("darkgrid")

In [ ]:
ads_docs = gasdb.get_adsorption_docs()

In [ ]:
energies = [doc["energy"] for doc in ads_docs]

In [ ]:
import pandas as pd

In [ ]:
energy_df = pd.DataFrame(data=ads_docs)

In [ ]:
# 07 Jul 2020: find out the slab metal. Because we're only looking
# at pure Pt and Ru slabs right now, we can pull this from the
# adsorbate coordination string.

energy_df["near_slab_element"] = energy_df["coordination"].apply(
    lambda x: x.split("-")[0]
)

In [ ]:
len(energy_df)

In [ ]:
# Swarm plot
fig, ax = plt.subplots(1, 1, figsize=(8, 3))
sns.swarmplot(
    data=energy_df,
    y="energy",
    x="adsorbate",
    palette="Dark2",
    hue="near_slab_element",
    s=3,
    ax=ax,
)

# Label axes
ax.set_xlabel("Adsorbate")
ax.set_ylabel("Binding energy / eV")

# Change legend title
lg = ax.get_legend()
lg.set_title("First coord. slab metal")

datestr = datetime.date.today().strftime("%d %b %Y")
ax.set_title(f"Preliminary adsorption energies, {datestr}")
fig.tight_layout()

fig.savefig(f"""ads-energies-swarm-plot_{datestr.replace(" ","-").lower()}.pdf""")

In [ ]:
# Violin plot
fig, ax = plt.subplots(1, 1, figsize=(15, 8))
sns.violinplot(data=energy_df, y="energy", x="adsorbate", width=1, scale="count", ax=ax)
# ax.set_xticks([0, 0.5, 1, 1.5])

In [ ]:
# Inspect why some O adslabs have relatively high binding energies
import gaspy.mongo as gaspymongo

high_docs = [doc for doc in ads_docs if doc["energy"] > 1]

high_docs = gasdb.get_adsorption_docs(
    adsorbate="O",
    extra_projections={"atoms": "$atoms", "results": "$results"},
    filters={"adsorption_energy": {"$gt": 1.00}},
)



high_atoms = [gaspymongo.make_atoms_from_doc(doc) for doc in high_docs]

view(high_atoms)

In [ ]:
# Compare to low-energy O adslabs
import gaspy.mongo as gaspymongo

low_docs = gasdb.get_adsorption_docs(
    adsorbate="O",
    extra_projections={"atoms": "$atoms", "results": "$results"},
    filters={"adsorption_energy": {"$lt": 1.00}},
)



low_atoms = [gaspymongo.make_atoms_from_doc(doc) for doc in low_docs]

view(low_atoms)

### Defuse fireworks that match a query

- Allows you to defuse (to archive and not run) Fireworks based on querying the Fireworks database.
- WARNING: don't run this unless you're absolutely sure that you don't want to run these calculations again. If you do, you'll need to reignite the fireworks, or use GASpy Luigi jobs to resubmit them.

In [ ]:
lpad = fireworks_helper_scripts.get_launchpad()

In [ ]:
fws_to_defuse = lpad.get_fw_ids(query={"name.calculation_type": "unit cell optimization", "state": "READY"})

In [ ]:
fws_to_defuse

In [ ]:
for fwid in fws_to_defuse:
    lpad.defuse_fw(fw_id=fwid)

In [ ]:
lpad.defuse_fw(190)

## Maintenance

- This section has tasks for maintaining the database and other parts of the system.

### Back up Mongo database

- We should probably make this a cron job, but here's the template the Ulissi group sent us.

In [ ]:
import pymongo

In [ ]:
from gaspy.utils import read_rc

In [ ]:
# Read variables
mongo_host = read_rc("fireworks_info.lpad.host")
mongo_port = read_rc("fireworks_info.lpad.port")
username = read_rc("fireworks_info.lpad.username")
password = read_rc("fireworks_info.lpad.password")

backup_location = read_rc("fireworks_info.backup_directory")

now = datetime.date.today().strftime("%Y_%m_%d")

In [ ]:
command = (
    f"""module load mongodb; mongodump --host "{mongo_host}" --port "{mongo_port}" """
    f"""--username "{username}" --password "{password}" """
    f"""--out "{os.path.join(backup_location, "mongodb_backup_" + now)}" """
    f"""--db "goldsmith-gaspy" --gzip"""
)

In [ ]:
result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, encoding="utf8", shell=True)

In [ ]:
print(result.stderr)